# Note on the stuff done in this notebook
Date: 11th Nov 2020 

1# In this notebook, we will use the GCP Cloud NLP API to find sentiments for Dutch tweets. This is part of the work given to the Amsterdam research group. 

Reading the excel files in pandas is causing a problem (due to some unnatural characters in the text files!! :D) 

So we are directly reading the Google Spreadsheet files.

2# Next I realised that the Dutch sentiment analyzer was not working correctly. However the translated text was working okay. So if I want to analyze a Dutch tweet, I have to first translate it to English & then do the sentiment analysis

Tried writing to a google sheet, (using gspread) but it didn't work because it failed due to the special character

3# Then used the gspread utility to write a Dutch tweet and then translating it using the GOOGLETRANSLATE utility from GoogleSheets

4# (21-Nov-2020) Updated the sheet to check for sentiment scores for all the available languages



---
# Install & import all the required packages


In [ ]:
!pip install --upgrade google-cloud-language

In [ ]:
!pip install google-cloud-translate==2.0.1

In [ ]:
!pip3 install gspread

In [ ]:
# Imports the Google Cloud client library
from google.cloud import language_v1

import os
import pandas as pd
import numpy as np
from google.colab import drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




---
# Instantiate the env variables


In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/API_Key.json"

In [ ]:
# Instantiates a client
client = language_v1.LanguageServiceClient()

In [ ]:
lang_support = ['zh', 'zh-Hant', 'en', 'fr', 'de', 'it', 'ja', 'ko', 'pt', 'ru', 'es' ]



---

# Sentiment analysis service

In [ ]:
# The text to analyze
text = u"RT @ deedeedoodar1: The food was ridiculously bad"
document = language_v1.Document(content=text, language = "en", type_=language_v1.Document.Type.PLAIN_TEXT)

In [ ]:
# Detects the sentiment of the text
sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment

In [ ]:
print("Text: {}".format(text))
print("Sentiment: {}, {}".format(sentiment.score, sentiment.magnitude))

Text: RT @ deedeedoodar1: The food was ridiculously bad
Sentiment: -0.800000011920929, 0.800000011920929


In [ ]:
#!ls "drive/My Drive/Dutch Tweets/Tweets Excel_v3"

In [ ]:
#!cp "drive/My Drive/Dutch Tweets/Tweets Excel_v3/allData_chunk5.xlsx" .

In [ ]:
#import pandas as pd

# NOTE:: reading directly via pandas was causing an error due to special chars in the tweet text 

In [ ]:
#gc = gspread.service_account() << This was giving an error of gspread not defined



---
# Read data from Google Sheets and Update the sample tweets in Dutch translation (sentiment)


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
wks = gc.open("allData_chunk5").sheet1 # You dont have to givve the path to the gsheet!!! 

In [ ]:
allData5 = pd.DataFrame(wks.get_all_records())
print(allData5.shape)

(27104, 20)


In [ ]:
sample = allData5.sample(100).reset_index()
print(sample.shape)
sample.head()

(100, 21)


,index,full_text,text Translation,created_at,screen_name,description,desc Translation,location,Weekday,Month,Year,Day,point_info,point,latitude,longitude,altitude,Province,hisco_standard,hisco_code,Industry
0,8778,RT @MDHeeremans: U moet zich constructief opst...,RT @MDHeeremans: You must establish constructi...,Tue Sep 01 13:09:01 +0000 2020,natalia4u2,Love studie Bedrijfseconomie❤ Muay Thai❤Bikram...,Love Bedrijfseconomie❤ study Muay Thai❤Bikram ...,"Amsterdam, Nederland",1,9,2020,1,"Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.3728,4.8936,0,Noord-Holland,,,FALSE
1,22917,"RT @fvdemocratie: Géén permanente verandering,...","RT @fvdemocratie: No permanent change, not a n...",Wed May 20 21:26:01 +0000 2020,DoornGijs,,,,2,5,2020,20,,,,,0,FALSE,,,FALSE
2,25146,RT @Soundsenz: Miskien moet sigaretmaatskappye...,RT @Soundsenz: miskien must maatskappye cigare...,Sat Jun 27 06:46:25 +0000 2020,RootzLife,"Avon & Justine Rep, Administrator, Loves music...","Avon & Justine Rep, Administrator, Loves Music...",South Africa,5,6,2020,27,South Africa,"(-28.8166236, 24.991639, 0.0)",-28.8166,24.9916,0,FALSE,,,FALSE
3,25709,@trotsopdeboer Als Nederland goed uit deze cri...,@trotsopdeboer Netherlands If good comes out o...,Fri Apr 03 09:40:26 +0000 2020,come2twit,"Mundus vult decipi, ergo decipiatur\n\n🎶 It se...","Mundus fills decipi, ergo decipiatur\n\n🎶 It s...","s-Hertogenbosch, Nederland",4,4,2020,3,,,,,,,boer,61110,TRUE
4,6504,RT @georgevanhouts: De nu meer dan lachwekkend...,RT @georgevanhouts: The now more than risible ...,Sun Jun 07 18:26:06 +0000 2020,HansWinger,nondualiteit | complementaire geneeswijzen en ...,nonduality | complementary medicine and health...,,6,6,2020,7,,,,,0,FALSE,,,FALSE


In [ ]:
score = list()
magnitude = list()

for idx, row in sample.iterrows():
  # The text to analyze
  text = row["text Translation"]
  
  # Detects the sentiment of the text
  try:
    document = language_v1.Document(content=text, language ="en",type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    score.append(sentiment.score)
    magnitude.append(sentiment.magnitude)
  except: 
    print("Error: ", text)
    score.append(-1)
    magnitude.append(-1)
  
  

sample['Sentiment Score'] = score
sample['Sentiment MAgnitude'] = magnitude

In [ ]:
sample.head(15)

,index,full_text,text Translation,created_at,screen_name,description,desc Translation,location,Weekday,Month,Year,Day,point_info,point,latitude,longitude,altitude,Province,hisco_standard,hisco_code,Industry,Sentiment Score,Sentiment MAgnitude
0,18188,@NOS waarom is het Corona-liveblog gesloten?,@NOS why Corona live blog is closed?,Thu Mar 12 10:18:23 +0000 2020,bpenris,I like trains. And computers. And stuff. And i...,I like trains. And computers. And stuff. And i...,,3,3,2020,12,,,,,0,FALSE,,,FALSE,-0.5,0.5
1,1232,RT @luchtvaart: VAE verbiedt bijna alle vlucht...,RT @luchtvaart: UAE bans almost all flights to...,Mon Feb 03 21:56:40 +0000 2020,kjwoerkom,Luchtvaartjournalist | @luchtvaart | Luchtvaar...,Aviation Journalist | @luchtvaart | Aviation N...,Schiphol,0,2,2020,3,"Schiphol, Haarlemmermeer, Noord-Holland, Neder...","(52.3080392, 4.7621975, 0.0)",52.308,4.7622,0,Noord-Holland,,,FALSE,-0.5,0.5
2,20665,RT @MarcoZannoniCOT: Nieuwe tips voor gemeente...,RT @MarcoZannoniCOT: New Tips municipalities /...,Mon Mar 30 07:33:01 +0000 2020,CyrielHamstra,1973 - ❤️&💙💙💙 - lente & zomer ☀️ - beeld & gel...,1973 - ❤️ & 💙💙💙 - Spring & summer ☀️ - picture...,Zuidlaren,0,3,2020,30,"Zuidlaren, Tynaarlo, Drenthe, Nederland","(53.08584745, 6.6966895314804304, 0.0)",53.0858,6.69669,0,Drenthe,gemeente,31090,TRUE,0.0,0.0
3,26098,@kuifmees Testen geneest ook geen Corona.,@kuifmees Testing heals no Corona.,Tue Mar 10 10:58:25 +0000 2020,AlbertZvld,,,,1,3,2020,10,,,,,0,FALSE,,,FALSE,0.4,0.4
4,24459,"Geen ongemakkelijke zoenpartijen meer, geen vo...","No awkward kiss parties, no national anthem at...",Tue Aug 18 17:11:40 +0000 2020,fkwebeman,Deugende ingenieur. Man van een juf. Bezocht e...,Deug Ende engineer. Husband of a teacher. Visi...,,1,8,2020,18,,,,,0,FALSE,,,FALSE,0.0,0.2
5,1039,"RT @TimesEurope: U.S. whipping up panic, China...","RT @TimesEurope: U.S. whipping up panic, says ...",Mon Feb 03 21:39:34 +0000 2020,marco_kerkhofs,🔴#TERRORISM🔴#CHINA🔴🕌🕍🏥#ISLAM #ISRAEL #USA #EU🔴...,🔴 TERRORISM🔴 # # # CHINA🔴🕌🕍🏥 ISLAM #ISRAEL #US...,Amsterdam,0,2,2020,3,"Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.3728,4.8936,0,Noord-Holland,,,FALSE,-0.3,0.3
6,2999,RT @wierdduk: Afgelopen weekend in Londen: htt...,RT @wierdduk: Last weekend in London: https://...,Tue Sep 01 08:01:43 +0000 2020,VerkerkSok,,,,1,9,2020,1,,,,,0,FALSE,,,FALSE,0.0,0.0
7,5195,@joyce2104 @EenVandaag En men dat wil bereiken...,@ Joyce2104 @EenVandaag And it wants to achiev...,Mon Aug 31 18:49:15 +0000 2020,DrutenOtto,,,,0,8,2020,31,,,,,0,FALSE,,,FALSE,-0.1,0.7
8,8012,RT @mind_spin: Je kunt wel boven de wet gaan l...,RT @mind_spin: You can start living above the ...,Mon Aug 17 20:18:37 +0000 2020,appie1953,,,,0,8,2020,17,,,,,0,FALSE,min,7410,TRUE,0.0,0.0
9,22001,RT @ProSyn: The @IMFNews estimates that govern...,RT @ProSyn: The @IMFNews estimates That Govern...,Thu Sep 03 11:04:19 +0000 2020,macharoesink,as an art historian and curator involved in ar...,as an art historian and curator Involved in ar...,The Netherlands,3,9,2020,3,Nederland,"(52.5001698, 5.7480821, 0.0)",52.5002,5.74808,0,Flevoland,,,FALSE,-0.4,0.4




---

# Read cells from Google sheets

In [ ]:
import numpy as np 
np.mean(magnitude)

0.7220000011473894

In [ ]:
np.mean(score)

-0.12100000202655792

In [ ]:
sh = gc.create('Sample Dutch Sentiment Analysis')

In [ ]:
worksheet = sh.sheet1   #add_worksheet(title="Sheet 1", rows="120", cols="25")

In [ ]:
worksheet.insert_row(sample.columns.values.tolist())

{'spreadsheetId': '1WaLGGuecA4YxgubDBmnnUbBW0TWTA60EtSCTHdIRqaE',
 'updatedCells': 23,
 'updatedColumns': 23,
 'updatedRange': 'Sheet1!A1:W1',
 'updatedRows': 1}

In [ ]:
sample.to_excel('sample.xlsx')

In [ ]:
sample.columns.values.tolist()

['index',
 'full_text',
 'text Translation',
 'created_at',
 'screen_name',
 'description',
 'desc Translation',
 'location',
 'Weekday',
 'Month',
 'Year',
 'Day',
 'point_info',
 'point',
 'latitude',
 'longitude',
 'altitude',
 'Province',
 'hisco_standard',
 'hisco_code',
 'Industry',
 'Sentiment Score',
 'Sentiment MAgnitude']

Sample code for text translation (Dutch ->> English)

In [ ]:
os.environ["GOOGLE_CLOUD_PROJECT"]="project-x-262017"

In [ ]:
from google.cloud import translate_v2 as translate

translate_client = translate.Client()

In [ ]:
text = "Ik ben zo fucking klaar met so wel @PostNL als @mediamarkt_nl. Dus ik bestel woensdag dus een pakketje. Nou dat de morgen in huis met de corona niet lukt snap ik. Maar ik heb 2 keer een “komt morgen aan mail” gehad zonder dat er iets aan kwam -"
target = "en"
source = "nl"

In [ ]:
# Text can also be a sequence of strings, in which case this method
# will return a sequence of results for each text.
result = translate_client.translate(text, target_language=target)

print(u"Text: {}".format(result["input"]))
print(u"Translation: {}".format(result["translatedText"]))
print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

Text: Ik ben zo fucking klaar met so wel @PostNL als @mediamarkt_nl. Dus ik bestel woensdag dus een pakketje. Nou dat de morgen in huis met de corona niet lukt snap ik. Maar ik heb 2 keer een “komt morgen aan mail” gehad zonder dat er iets aan kwam -
Translation: I&#39;m so fucking done with @PostNL as @mediamarkt_nl. So I order a package on Wednesday. Well that the morning at home with the corona does not work I understand. But I had 2 times a &quot;mail coming tomorrow&quot; without anything coming -
Detected source language: nl


In [ ]:
result

{'detectedSourceLanguage': 'nl',
 'input': 'Ik ben zo fucking klaar met so wel @PostNL als @mediamarkt_nl. Dus ik bestel woensdag dus een pakketje. Nou dat de morgen in huis met de corona niet lukt snap ik. Maar ik heb 2 keer een “komt morgen aan mail” gehad zonder dat er iets aan kwam -',
 'translatedText': 'I&#39;m so fucking done with @PostNL as @mediamarkt_nl. So I order a package on Wednesday. Well that the morning at home with the corona does not work I understand. But I had 2 times a &quot;mail coming tomorrow&quot; without anything coming -'}

In [ ]:
len(text) # Check size of text for calculating the pricing for sentiment analysis of all the tweets

244

In [ ]:
275000/4000

68.75

In [ ]:
70*20

1400



---


# Setting up a sample pipeline for using google translation (using a cheat via google sheets)

In [ ]:
sh = gc.create('Google Translate')

In [ ]:
worksheet = sh.sheet1

In [ ]:
worksheet.update_cell(1,1, 'Ik ben zo fucking klaar met so wel @PostNL als @mediamarkt_nl. Dus ik bestel woensdag dus een pakketje. Nou dat de morgen in huis met de corona niet lukt snap ik.')

{'spreadsheetId': '1MIoDo4Gc3H76kqQwvGVc28drft7ZEkxQ5yNkd4noTpg',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!A1',
 'updatedRows': 1}

In [ ]:
worksheet.update_cell(1,2, '=googletranslate(a1, "nl", "en")') 

{'spreadsheetId': '1MIoDo4Gc3H76kqQwvGVc28drft7ZEkxQ5yNkd4noTpg',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!B1',
 'updatedRows': 1}

In [ ]:
worksheet.cell(1,2).value

"I'm so fucking done so well as @PostNL @mediamarkt_nl. So I order Wednesday so a package. Well that morning in the house with the corona I do not succeed."



---
# Reading the sample files and check sentiment for various languages


In [ ]:
wks = gc.open("Sample Dutch Sentiment Analysis").worksheet("Sheet2")

In [ ]:
allText = pd.DataFrame(wks.get_all_records())
print(allText.shape)
allText.head(34)

(32, 6)


,Base Sentiment,Base Statement,Language,Translation,Sentiment Score,Sentiment Magnitude
0,Negative,The food was ridiculously bad,ar,كان الطعام سيئا يبعث على السخرية,-0.5,0.5
1,Negative,The food was ridiculously bad,zh,食品是坏得离谱,-0.7,0.7
2,Negative,The food was ridiculously bad,zh-Hant,食品是壞得離譜,-0.8,0.8
3,Negative,The food was ridiculously bad,nl,Het eten was belachelijk slecht,0.0,0.0
4,Negative,The food was ridiculously bad,en,The food was ridiculously bad,-0.8,0.8
5,Negative,The food was ridiculously bad,fr,La nourriture était ridiculement mauvaise,-0.9,0.9
6,Negative,The food was ridiculously bad,de,Das Essen war unglaublich schlecht,-0.5,0.5
7,Negative,The food was ridiculously bad,id,Makanan yang ridiculously buruk,-0.5,0.5
8,Negative,The food was ridiculously bad,it,Il cibo era incredibilmente male,-0.9,0.9
9,Negative,The food was ridiculously bad,ja,食べ物は途方もなく悪かったです,-0.8,0.8


In [ ]:
def checkSentiment(text, lang):
  try:
    document = language_v1.Document(content=text, language =lang, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    return sentiment.score, sentiment.magnitude

  except:
    print(f"ERROR: For text: {text}, in leng: {lang}")
    return None, None

In [ ]:
senti = allText.apply( lambda row: checkSentiment(row['Translation'], row['Language']), axis=1)

In [ ]:
allText['Sentiment Score'] = senti.apply(lambda x: x[0])
allText['Sentiment Magnitude'] = senti.apply(lambda x: x[1])

In [ ]:
allText

,Base Sentiment,Base Statement,Language,Translation,Sentiment Score,Sentiment Magnitude
0,Negative,The food was ridiculously bad,ar,كان الطعام سيئا يبعث على السخرية,-0.5,0.5
1,Negative,The food was ridiculously bad,zh,食品是坏得离谱,-0.7,0.7
2,Negative,The food was ridiculously bad,zh-Hant,食品是壞得離譜,-0.8,0.8
3,Negative,The food was ridiculously bad,nl,Het eten was belachelijk slecht,0.0,0.0
4,Negative,The food was ridiculously bad,en,The food was ridiculously bad,-0.8,0.8
5,Negative,The food was ridiculously bad,fr,La nourriture était ridiculement mauvaise,-0.9,0.9
6,Negative,The food was ridiculously bad,de,Das Essen war unglaublich schlecht,-0.5,0.5
7,Negative,The food was ridiculously bad,id,Makanan yang ridiculously buruk,-0.5,0.5
8,Negative,The food was ridiculously bad,it,Il cibo era incredibilmente male,-0.9,0.9
9,Negative,The food was ridiculously bad,ja,食べ物は途方もなく悪かったです,-0.8,0.8


In [ ]:
for idx, row in allText.iterrows():
  try:
    wks.update_cell(idx+2, 5, row['Sentiment Score'])
    wks.update_cell(idx+2, 6, row['Sentiment Magnitude'])
  except:
    wks.update_cell(idx+2, 5, "")
    wks.update_cell(idx+2, 6, "")

In [ ]:
allText.to_excel("Sample.xlsx")